# Introduction

## General setup

Import some general stuff, set up the sagemaker environment

In [ ]:
import os
import pandas as pd
import numpy as np

In [14]:
import sagemaker
from sagemaker.session import Session
import boto3

session = Session()
role = sagemaker.get_execution_role()
bucket = session.default_bucket()
bucket_folder = "numerai"
prefix = os.path.join(bucket_folder, "explore")

## Data

Second step is formatting the data, splitting it in several batches and splitting it for training, validation and testing

In [21]:
training_data = pd.read_csv("data/numerai_training_data.csv")
tournament = pd.read_csv("data/numerai_tournament_data.csv")

MemoryError: 

In [ ]:
training_data.head()

## Models 

The third step is setting up the models. We define general parameters as well as each model we want to test

In [17]:
models = []
# Set some defaults so we don't need to type this every time
training_instance_count = 1
training_instance_type = "ml.m4.xlarge"

model_kwargs = {
    "role": role,
    "train_instance_count": training_instance_count,
    "train_instance_type": training_instance_type,
    "output_path": prefix,
    "sagemaker_session": session,
    
}

In [19]:
linear_learner = sagemaker.LinearLearner(
    predictor_type="regressor", #Also try multiclass?
    epochs=10,
    optimizer="auto", #Can also choose sgd, adam or rmsprop
    loss="auto", #Can also choose ‘logistic’, ‘squared_loss’, ‘absolute_loss’, ‘hinge_loss’, ‘eps_insensitive_squared_loss’, ‘eps_insensitive_absolute_loss’, ‘quantile_loss’, ‘huber_loss’ 
    learning_rate=None,
    **model_kwargs,
)
models.append(linear_learner)
# For multiclass there is also accuracy_top_k, f_beta and balance_multiclass_weights 

In [20]:
linear_learner_hyperparameters = {}

## Training

# Scoring